
## Introduction

This repository contains code for a demonstration of using Retrieval-Augmented Generation (RAG) for generating advertisements targeting travelers. The code utilizes Pinecone vector database for similarity search, Sentence Transformers for generating embeddings, and Replicate for fine-tuning and generating text based on the provided context.

## Requirements

Before running the code, make sure you have the following dependencies installed:

- Python 3.x
- Pinecone
- Sentence Transformers
- Replicate


## Usage

1. Set up Pinecone: Obtain an API key from Pinecone and replace the empty string in the code with your API key:

    ```python
    pc = Pinecone(api_key="YOUR_API_KEY_HERE")
    index = pc.Index("tourist-index")
    ```

2. Set up Replicate: Set your Replicate API token as an environment variable:

    ```bash
    export REPLICATE_API_TOKEN="YOUR_API_TOKEN_HERE"
    ```

3. Run the code: Execute the Python script or Jupyter notebook containing the provided code snippets.

## Functionality

1. **Similarity Search**: The code performs similarity search using Pinecone, allowing you to find relevant context based on a given query.

2. **Text Embedding**: Sentence Transformers are used to generate embeddings for the input text.

3. **Text Generation**: Replicate is utilized for generating engaging advertisements targeted at travelers based on the context retrieved from the similarity search.

## License

This code is provided under the [MIT License](LICENSE). Feel free to modify and distribute it according to your needs.

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
from pinecone import Pinecone
from pinecone import ServerlessSpec, PodSpec
import os

pc = Pinecone(api_key="YOUR_API_KEY_HERE")
index = pc.Index("tourist-index")

In [35]:
# small helper function so we can repeat queries later
def run_query(query):
    embedding = create_vector(query)
    results = index.query(top_k=3, vector=embedding, include_metadata=True, include_values=False)
    context = []
    for result in results['matches']:
        if round(result['score'], 2) > 0.5:
            context.append(result['metadata']['text'])
    return "\n".join(context)

In [15]:
from sentence_transformers import SentenceTransformer
import torch

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def create_vector(query, n_results=3):
    
    return embedding_model.encode(query).tolist()

In [38]:
query = "I like to visit historical places what are my options?"
context = run_query(query)

In [4]:
import replicate
import os

In [5]:
os.environ["REPLICATE_API_TOKEN"] = "YOUR_API_TOKEN_HERE"

In [71]:
from langchain.prompts import PromptTemplate

prompt_template = f"""[INST] <<SYS>>\
Use the context to generate an interesting and captivating advertisement to travelers.
Context: {context}
<</SYS>>
{query}[/INST]"""

# Create prompt from prompt template 
prompt_template

"[INST] <<SYS>>Use the context to generate an interesting and captivating advertisement to travelers.\nContext: Step back in time and explore the rich history of London, one of the world's most iconic cities. Visit historic landmarks like the Tower of London, Buckingham Palace, and Westminster Abbey, where you can trace the footsteps of kings, queens, and political leaders. Explore the city's world-class museums and galleries, including the British Museum, Tate Modern, and National Gallery, which showcase treasures from around the world. Wander through charming neighborhoods like Covent Garden and Notting Hill, where you'll find historic pubs, quaint cafes, and bustling markets. With its blend of ancient traditions and modern innovation, London offers a wealth of historical sites and cultural experiences for visitors to enjoy.\n<</SYS>>\nI like to visit historical places what are my options?[/INST]"

In [74]:
output = replicate.stream(
  "meta/llama-2-70b-chat",
  input={
        "debug": False,
#         "top_k": 50,
#         "top_p": 1,
#         "temperature": 0.75,
        "max_new_tokens": 500,
        "prompt": prompt_template,
    },
)
for s in output:
    print(s, end="", flush=False)






Are you ready to step back in time and explore the rich history of one of the world's most iconic cities? Look no further than London, a city that seamlessly blends ancient traditions with modern innovation.

Immerse yourself in the city's storied past by visiting historic landmarks like the Tower of London, Buckingham Palace, and Westminster Abbey. These majestic buildings have witnessed countless significant events throughout history, and offer visitors a chance to trace the footsteps of kings, queens, and political leaders.

But London's historical significance doesn't stop at its famous landmarks. The city is also home to an array of world-class museums and galleries, showcasing treasures from around the world. Discover artifacts from ancient civilizations at the British Museum, marvel at contemporary art at Tate Modern, or admire some of the greatest works of art in human history at the National Gallery.

When you need a break from the hustle and bustle of sightseeing, wander